<a href="https://colab.research.google.com/github/fabriciosantana/mcdia/blob/main/05-iag/nemotron_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA Nemotron - Exemplos de Uso

O **Nemotron** é uma família de modelos da NVIDIA com arquitetura híbrida Mamba-Transformer MoE.

Este notebook mostra como usar o Nemotron via NVIDIA NIM API.

## 1. Configurar API Key


In [2]:
import os
from google.colab import userdata

def get_nvidia_api_key():
    key = None
    if key:
        key = os.environ.get("NVIDIA_API_KEY")
    else:
        key = userdata.get('NVIDIA_API_KEY')

    return key

NVIDIA_API_KEY = get_nvidia_api_key()

if NVIDIA_API_KEY:
    print(f"✅ NVIDIA_API_KEY carregada: {NVIDIA_API_KEY[:15]}...")
else:
    print("⚠️ NVIDIA_API_KEY não encontrada. Defina em ~/.bashrc ou manualmente:")
    print('os.environ["NVIDIA_API_KEY"] = "sua_chave_aqui"')

✅ NVIDIA_API_KEY carregada: nvapi-8tdO1NjXD...


## 2. Testar conexão com NVIDIA API

In [ ]:
import requests

invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "nvidia/nemotron-3-nano-30b-a3b",
    "messages": [
        {"role": "user", "content": "Explique o que é Deep Learning em 3 frases."}
    ],
    "temperature": 0.7,
    "max_tokens": 200
}

response = requests.post(invoke_url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print("✅ Resposta do Nemotron:")
    print(result["choices"][0]["message"]["content"])
else:
    print(f"❌ Erro {response.status_code}: {response.text}")

## 3. Usando a biblioteca OpenAI (compatível)

In [ ]:
# pip install openai
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[
        {"role": "user", "content": "Qual a diferença entre RNN e Transformer?"}
    ],
    temperature=0.7,
    max_tokens=300
)

print(response.choices[0].message.content)

## 4. Streaming (resposta em tempo real)

In [ ]:
print("🤖 Nemotron: ", end="")

for chunk in client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Liste 3 aplicações práticas de LLMs."}],
    temperature=0.7,
    max_tokens=200,
    stream=True
):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

## 5. Chat com contexto (múltiplas mensagens)

In [ ]:
messages = [
    {"role": "system", "content": "Você é um especialista em IA. Seja técnico e conciso."},
    {"role": "user", "content": "O que é atenção em transformers?"},
]

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=messages,
    temperature=0.7
)

print(response.choices[0].message.content)

## 6. Parâmetros avançados

In [ ]:
response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Gere um título criativo para um artigo sobre IA."}],
    temperature=0.9,
    max_tokens=50,
    top_p=0.9
)

print(response.choices[0].message.content)

## Modelos Nemotron Disponíveis

| Modelo | Tamanho | Descrição |
|--------|---------|----------|
| `nvidia/nemotron-3-nano-30b-a3b` | 30B (3.5B ativos) | MoE híbrido, reasoning |
| `nvidia/nemotron-4-340b-instruct` | 340B | Modelo grande para tarefas complexas |

### Características:
- **Arquitetura**: Mamba2-Transformer Hybrid MoE
- **Idiomas**: Inglês, Alemão, Espanhol, Francês, Italiano, Japonês
- **Capacidades**: Reasoning, tool calling, chat, código